In [22]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})





**1- OBJETIVOS DEL ESTUDIO**
- General: desarrollar un sistema de recomendación personalizado para cada usuario.
- Específicos: 

  1- identificar patrones de calificaciones y reseñas para comprender las preferencias de cada usuario, basado en el historial de calificaciones y reseñas.

  2- detectar libros con alta preferencia y calificaciópn pero pocas reseñas o visibilidad para promocionarlos

  3- analizar la relación que existe entre autores, editoriales y venta de libros, considerando calificaciones y número de reseñas.
  
  4- preparar los datos necesarios para entrenar los modelos que serán parte del sistema de recomendación personalizada

**2- ESTUDIO DE LAS TABLAS**

In [33]:
#Conexión a tabla books
query = "SELECT * FROM books LIMIT 5;"
df = pd.read_sql(query, engine)
display(df.head())


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [32]:
#Conexión a tabla authors
query = "SELECT * FROM authors LIMIT 5;"
df = pd.read_sql(query, engine)
display(df.head())

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [31]:
#Conexión a tabla publishers
query = "SELECT * FROM publishers LIMIT 5;"
df = pd.read_sql(query, engine)
display(df.head())

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [30]:
#Conexión a tabla ratings
query = "SELECT * FROM ratings LIMIT 5;"
df = pd.read_sql(query, engine)
display(df.head())

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [29]:
#Conexión a tabla reviews
query = "SELECT * FROM reviews LIMIT 5;"
df = pd.read_sql(query, engine)
display(df.head())

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**3- Número de libros publicados después del 1 de enero de 2000**


In [ ]:
query = """
SELECT COUNT(*) AS total_libros
FROM books
WHERE publication_date > '2000-01-01';
"""

resultado = pd.read_sql_query(query, engine)

display(resultado)

,total_libros
0,819


**4- Número de reseñas de usuarios y la calificación promedio para cada libro**

En este caso se necesita calcular, para cada libro: el número de reseña de la tabla 'reviews' y la calificación promedio de la tabla 'ratings'.Se debe considerar que un libro puede tener varias reseñas y varias calificaciones y que ambs se encuentran en tablas seperadas, además, algunos libros pueden no tener reseñas o calificaciones, por lo que se debe utilizar LEFT JOIN.

Primero, se hará agrupación por libro 'book_id', 'title' y se contarán las reseñas utilizando la tabla 'reviews'.Aquí se usa LEFT JOIN para incluir libros sin reseñas o calificaciones



In [ ]:
# consulta SQL para obtener estadísticas por libro
query = """
SELECT 
    b.book_id,
    b.title,
    a.author,
    p.publisher,
    COUNT(DISTINCT rv.review_id) AS numero_reseñas,
    ROUND(AVG(rt.rating), 2) AS calificacion_promedio,
    COUNT(DISTINCT rt.rating_id) AS total_calificaciones
FROM books b
JOIN authors a ON b.author_id = a.author_id
JOIN publishers p ON b.publisher_id = p.publisher_id
LEFT JOIN reviews rv ON b.book_id = rv.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title, a.author, p.publisher
ORDER BY numero_reseñas DESC, calificacion_promedio DESC;
"""

#  consulta y resultados
resultados = pd.read_sqSl_query(query, engine)
print("\nEstadísticas de reseñas y calificaciones por libro:")
display(resultados)




Estadísticas de reseñas y calificaciones por libro:


,book_id,title,author,publisher,numero_reseñas,calificacion_promedio,total_calificaciones
0,948,Twilight (Twilight #1),Stephenie Meyer,Little Brown and Company,7,3.66,160
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,Scholastic Inc.,6,4.41,82
2,299,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré,Arthur A. Levine Books / Scholastic Inc.,6,4.29,80
3,656,The Book Thief,Markus Zusak/Cao Xuân Việt Khương,Alfred A. Knopf,6,4.26,53
4,734,The Glass Castle,Jeannette Walls,Scribner,6,4.21,29
...,...,...,...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,Teddy Slater/Ron Dias/Ric González,Golden/Disney,0,4.00,1
996,387,Leonardo's Notebooks,Leonardo da Vinci/H. Anna Suh,Black Dog & Leventhal,0,4.00,2
997,221,Essential Tales and Poems,Edgar Allan Poe/Benjamin F. Fisher,Barnes Noble Classics,0,4.00,3
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,Faye Perozich/Anne Rice/John Bolton/Daerick Gröss,Ballantine Books,0,3.67,3


**5- Editorial que ha publicado el mayor número de libros con más de 50 páginas**

Para esto primero se debe filtrar la tabla 'books' por su columna 'num_pages' y poner como condición que este valor sea mayor a 50.Luego, se agrupa por 'publisher_id' y se realiza el conteo de los libros, ordenando este conteo de manera descendente.Para obtener el nombre de la editorial, se debe hacer unión con la tabla 'publishers' 

In [ ]:
# consulta SQL para identificar la editorial con más libros >50 páginas
query = """
WITH libros_largos AS (
    SELECT publisher_id, COUNT(*) AS total_libros
    FROM books
    WHERE num_pages > 50
    GROUP BY publisher_id
)
SELECT p.publisher_id, p.publisher, ll.total_libros
FROM libros_largos ll
JOIN publishers p ON ll.publisher_id = p.publisher_id
WHERE ll.total_libros = (SELECT MAX(total_libros) FROM libros_largos)
ORDER BY ll.total_libros DESC;
"""

#  consulta y  resultados
resultados = pd.read_sql_query(query, engine)
print("\nEditorial(es) con más libros de más de 50 páginas:")
display(resultados)

# libros relevantes para verificación
query_books = """
SELECT b.book_id, b.title, b.num_pages, p.publisher
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
ORDER BY p.publisher, b.num_pages DESC;
"""

libros = pd.read_sql_query(query_books, engine)
print("\nLibros con más de 50 páginas:")
display(libros)


Editorial(es) con más libros de más de 50 páginas:


,publisher_id,publisher,total_libros
0,212,Penguin Books,42



Libros con más de 50 páginas:


,book_id,title,num_pages,publisher
0,815,The Once and Future King (The Once and Future ...,639,Ace
1,600,Stranger in a Strange Land,528,Ace
2,272,Furies of Calderon (Codex Alera #1),504,Ace
3,415,Magic Bites (Kate Daniels #1),261,Ace
4,595,Starship Troopers,335,Ace Book
...,...,...,...,...
987,339,I Capture the Castle,343,Wyatt Book
988,401,Long Day's Journey into Night,179,Yale University Press
989,692,The Cricket in Times Square,134,Yearling
990,457,My Father's Dragon (My Father's Dragon #1),96,Yearling


Se ha identificado a la editorial más prolífica en términos de publicar libros extensos, lo que puede ser útil para estrategias de marketing, acuerdos con autores o análisis respecto de tendencias editoriales.

**6- Autor con más alta calificación promedio**

Primero se deben encontrar aquellos libros que tienen al menos 50 calificaciones, luego, se calcula la calificación promedio para cada uno de ellos.Luego, por cada autor, se calcula la calificación promedio de todos aquellos de sus libros que cumplen con el criterio inicial y cuenta cuántos libros cumplen con el criterio.Esto identificará al autor con la más alta calificación promedio.
Finalmente, se hace una tabla de verificación que muestra calificaciones individuales por cada libro considerando también las más de 50 calificaciones.

In [55]:
# Consulta SQL para identificar al autor con la más alta calificación promedio
query = """
WITH libros_con_mas_de_50_calificaciones AS (
    SELECT 
        b.book_id,
        b.author_id,
        b.title,
        COUNT(r.rating_id) AS total_calificaciones,
        AVG(r.rating) AS calificacion_promedio
    FROM books b
    JOIN ratings r ON b.book_id = r.book_id
    GROUP BY b.book_id, b.author_id, b.title
    HAVING COUNT(r.rating_id) >= 50
)
SELECT 
    a.author_id,
    a.author,
    ROUND(AVG(libro.calificacion_promedio), 2) AS calificacion_promedio_autor,
    COUNT(libro.book_id) AS libros_calificados
FROM libros_con_mas_de_50_calificaciones libro
JOIN authors a ON libro.author_id = a.author_id
GROUP BY a.author_id, a.author
ORDER BY calificacion_promedio_autor DESC
LIMIT 1;
"""

# consulta y  resultados
resultado = pd.read_sql_query(query, engine)
print("\nAutor con la más alta calificación promedio en libros con al menos 50 calificaciones:")
display(resultado)

#  detalles de los libros considerados
query_detalle = """
SELECT 
    b.book_id,
    b.title,
    a.author,
    COUNT(r.rating_id) AS total_calificaciones,
    ROUND(AVG(r.rating), 2) AS calificacion_promedio
FROM books b
JOIN ratings r ON b.book_id = r.book_id
JOIN authors a ON b.author_id = a.author_id
GROUP BY b.book_id, b.title, a.author
HAVING COUNT(r.rating_id) >= 50
ORDER BY calificacion_promedio DESC;
"""

detalle = pd.read_sql_query(query_detalle, engine)
print("\nDetalle de libros con al menos 50 calificaciones:")
display(detalle)



Autor con la más alta calificación promedio en libros con al menos 50 calificaciones:


,author_id,author,calificacion_promedio_autor,libros_calificados
0,236,J.K. Rowling/Mary GrandPré,4.28,4



Detalle de libros con al menos 50 calificaciones:


,book_id,title,author,total_calificaciones,calificacion_promedio
0,302,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,82,4.41
1,722,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. Tolkien,74,4.39
2,299,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré,80,4.29
3,656,The Book Thief,Markus Zusak/Cao Xuân Việt Khương,53,4.26
4,300,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,73,4.25
5,399,Little Women,Louisa May Alcott,52,4.19
6,301,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,75,4.19
7,750,The Hobbit or There and Back Again,J.R.R. Tolkien,88,4.13
8,779,The Lightning Thief (Percy Jackson and the Oly...,Rick Riordan,62,4.08
9,405,Lord of the Flies,William Golding,71,3.90


Este análisis muestra al autor con la calificación promedio más alta, considerando solo libros con suficiente participación o calificaciones de parte de los lectores para garantizar cierta significancia estadística.

**7- Número promedio de reseñas de texto entre los ususarios que calificaron más de 50 libros**

En primer lugar, se deben identificar los usuarios que han calificado más de 50 libros, es decir, que tienen más de 50 registros en la tabla 'ratings'.Para estos ususarios, se debe contar el número de reseñas de texto, es decir, rgitros en la tabla 'reviews'.Por último, se calcula el promedio de esas reseñas de texto por usuario.


In [56]:
# Consulta SQL para calcular el promedio de reseñas de texto
query = """
WITH usuarios_activos AS (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(*) > 50
),
reseñas_por_usuario AS (
    SELECT 
        ua.username,
        COUNT(rv.review_id) AS total_reseñas
    FROM usuarios_activos ua
    LEFT JOIN reviews rv ON ua.username = rv.username
    GROUP BY ua.username
)
SELECT 
    ROUND(AVG(total_reseñas), 2) AS promedio_reseñas
FROM reseñas_por_usuario;
"""

# consulta y  resultados
resultado = pd.read_sql_query(query, engine)
print("\nNúmero promedio de reseñas de texto entre usuarios que calificaron más de 50 libros:")
display(resultado)

# Mostrar detalles de los usuarios activos
query_detalle = """
SELECT 
    r.username,
    COUNT(DISTINCT rt.rating_id) AS total_calificaciones,
    COUNT(DISTINCT rv.review_id) AS total_reseñas
FROM ratings rt
JOIN (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(*) > 50
) r ON rt.username = r.username
LEFT JOIN reviews rv ON rt.username = rv.username
GROUP BY r.username;
"""

detalle = pd.read_sql_query(query_detalle, engine)
print("\nDetalle de usuarios activos:")
display(detalle)


Número promedio de reseñas de texto entre usuarios que calificaron más de 50 libros:


,promedio_reseñas
0,24.33



Detalle de usuarios activos:


,username,total_calificaciones,total_reseñas
0,jennifermiller,53,25
1,martinadam,56,27
2,paul88,56,22
3,richard89,55,26
4,sfitzgerald,55,28
5,xdavis,51,18


Este resultado muestra el nivel de engagement de los usuarios más activos, es decir, aquellos que traducen sus calificaciones en reseñas de texto.En promedio, aquellos usuarios que califican más de 50 libros, hacen 24.33 reseñas cada uno.